In [22]:
import numpy as np
import pandas as pd
from scipy.special import erf
import ipynb

from ipynb.fs.defs.functions import *
from ChargeEquilibration import *

In [23]:
DATAPATH = "/home/alzbetak/Documents/projects/eem/mock-data/"

In [24]:
data = pd.read_csv(DATAPATH + "params_marina.dat", delim_whitespace=True)
data = data.iloc[5:] # drop first 5 lines
data

,#iac,atnm,sig,rng,eps,rng.1,hrd,rng.2,eln,rng.3,sig_2,rng.4,eps_2,rng.5
5,13,CH0,0.663949,0.0,0.006995,0.0,10.00000,0.0,6.00000,0.0,0.663949,0.0,0.006995,0.0
6,14,CH1,0.501918,0.0,0.094889,0.0,10.00000,0.0,6.00000,0.0,0.501918,0.0,0.094889,0.0
7,15,CH2,0.407038,0.0,0.410542,0.0,10.00000,0.0,6.00000,0.0,0.407038,0.0,0.410542,0.0
8,16,CH3,0.374792,0.0,0.867150,0.0,10.00000,0.0,8.00000,0.0,0.374792,0.0,0.867150,0.0
9,68,CX,0.372167,0.5,1.098306,0.5,12.79086,1.0,7.95720,0.5,0.374792,0.0,0.867150,0.0
10,69,CX2,0.389777,0.5,0.440980,0.5,9.98845,1.0,6.33650,0.5,0.407038,0.0,0.410542,0.0
11,39,CChl,0.435980,0.5,0.096146,0.5,7.92332,1.0,5.87687,0.5,0.501918,0.0,0.094889,0.0
12,45,CCl4,0.589737,0.5,0.006109,0.5,6.36747,1.0,5.96764,0.5,0.663949,0.0,0.006995,0.0
13,32,F,0.274332,0.5,0.697460,0.5,35.50866,1.0,18.71912,0.5,0.276700,0.0,0.637160,0.0
14,33,CL,0.338539,0.5,1.437321,0.5,24.45097,1.0,14.51889,0.5,0.340430,0.0,1.466040,0.0


In [25]:
# mock molecule: Cl3C-CH2-CH2-CFCl2
# index  0        1     2     3     4      5      6      7    8      9
atoms = ["CChl", "CL", "CL", "CL", "CH2", "CH2", "CChl", "F", "CL", "CL"]
bonds = [[0,1], [0,2], [0,3], [0,4], [4,5], [5,6], [6,7], [6,8], [6,9]]
bondLength = 0.1

In [26]:
# CHCl3
atoms = ["CChl", "CL", "CL", "CL"]
bonds = [[0,1], [0,2], [0,3]]
bondLength = 0.178

In [27]:
# index   0       1      2      3     4      5      6
atoms = ["CChl", "CL", "CH2", "CH2", "CH2", "CH2", "CH3"]
bonds = [[0,1], [0,2], [2,3], [3,4], [4,5], [5,6]]
bondLength = 0.18

In [28]:
# extract electronegativities, hardnesses and diameters
electronegativity = np.around([data.loc[data['atnm'] == a]["eln"].to_numpy()[0] for a in atoms], 5)
hardness = np.around([data.loc[data['atnm'] == a]["hrd"].to_numpy()[0] for a in atoms], 5)
diameters = np.around([data.loc[data['atnm'] == a]["sig"].to_numpy()[0] for a in atoms], 6)

# create connectivity matrix
connectivity = np.zeros((len(atoms), len(atoms)))

# add bonds
for b in bonds:
    connectivity[b[0],b[1]] = 1
    connectivity[b[1],b[0]] = 1
    
# higher-order connectivity 
while (len(np.where(connectivity == 0)[0]) > len(atoms)):
    tmpConnectivity = connectivity.copy()
    #loop over matrix
    for i in range(0,len(atoms)):
        neighborsI = [idx for idx,k in enumerate(connectivity[i]) if (k == 1)]
        #print("---" + str(neighborsI) + "---")
        for ni in neighborsI:
            neighborsJ = [idx for idx,k in enumerate(connectivity[ni]) if (k)]
            #print(neighborsJ)
            for nj in neighborsJ:
                if connectivity[i,nj] == 0 and i != nj:
                    order = connectivity[i,ni] + connectivity[ni,nj]
                    tmpConnectivity[i,nj] = order
                    tmpConnectivity[nj,i] = order

    connectivity = tmpConnectivity.copy()                

# mock distance matrix = 0.1 * connectivity
distanceMatrix = bondLength * connectivity

In [29]:
# parameters for all charge calculations
netCharge = 0
maxOrder = 2

# charge transfer topology: coupling over bonds
chargeTransferFilter = lambda x: 1 if (x == 1) else 0
chargeTransferTopology = np.vectorize(chargeTransferFilter)(connectivity)

In [30]:
# EEM
eem = EEM (connectivity=connectivity, distanceMatrix=distanceMatrix, diameters=diameters, hardness=hardness, \
           electronegativity=electronegativity, netCharge=netCharge, maxOrder=maxOrder)
charges = eem.compute()
eem.electronegativityEq

6.90507852655631

In [64]:
list(zip(atoms, np.around(eem.charges, 5)))

[('CChl', 0.18776),
 ('CL', -0.33431),
 ('CH2', 0.10297),
 ('CH2', 0.00679),
 ('CH2', 0.07748),
 ('CH2', 0.10291),
 ('CH3', -0.14359)]

In [44]:
pd.DataFrame(eem.JMatrix)

,0,1,2,3,4,5,6,7,8,9
0,7.923320,2.022076,2.022076,2.022076,1.874232,0.000000,0.000000,0.000000,0.000000,0.000000
1,2.022076,24.450970,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2.022076,0.000000,24.450970,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2.022076,0.000000,0.000000,24.450970,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1.874232,0.000000,0.000000,0.000000,10.000000,1.940680,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,1.940680,10.000000,1.874232,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,1.874232,7.923320,2.163356,2.022076,2.022076
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.163356,35.508660,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.022076,0.000000,24.450970,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.022076,0.000000,0.000000,24.450970


In [36]:
# Charge equilibration in atom space
qeq = QEqAtomic (connectivity=connectivity, distanceMatrix=distanceMatrix, diameters=diameters, hardness=hardness, \
           electronegativity=electronegativity, netCharge=netCharge, maxOrder=maxOrder)
charges = qeq.compute()

In [37]:
list(zip(atoms, np.around(qeq.charges, 5)))

[('CChl', 0.18776),
 ('CL', -0.33431),
 ('CH2', 0.10297),
 ('CH2', 0.00679),
 ('CH2', 0.07748),
 ('CH2', 0.10291),
 ('CH3', -0.14359)]

In [67]:
# Charge equilibration in bond space
bqeq = QEqBond (connectivity=connectivity, distanceMatrix=distanceMatrix, diameters=diameters, hardness=hardness, \
           electronegativity=electronegativity, netCharge=netCharge, maxOrder=maxOrder, chargeTransferTopology=chargeTransferTopology)
charges = bqeq.compute()

In [68]:
list(zip(atoms, np.around(bqeq.charges, 5)))

[('CChl', 0.18776),
 ('CL', -0.33431),
 ('CH2', 0.10297),
 ('CH2', 0.00679),
 ('CH2', 0.07748),
 ('CH2', 0.10291),
 ('CH3', -0.14359)]

In [69]:
bqeq.bondCharges

array([-0.3343141 ,  0.1465569 ,  0.0435871 ,  0.03679502, -0.0406817 ,
       -0.14358822])

In [70]:
pd.DataFrame(bqeq.bondJMatrix)

,0,1,2,3,4,5
0,28.426258,5.957201,-1.693057,1.685160,0.000000,0.000000
1,5.957201,14.251491,-7.951082,1.519053,-1.732057,0.000000
2,-1.693057,-7.951082,16.203674,-7.935730,1.565950,-1.732057
3,1.685160,1.519053,-7.935730,16.203674,-7.935730,1.618608
4,0.000000,-1.732057,1.565950,-7.935730,16.203674,-7.916930
5,0.000000,0.000000,-1.732057,1.618608,-7.916930,16.060757


In [71]:
# assume bond hardnesses = sum of atomic hardnesses
bVars = bqeq.bVars
bondHardness = np.zeros(len(bVars))
for b, [i, j] in enumerate(bVars):
    bondHardness[b] = (hardness[i] + hardness[j]) / 2

In [72]:
# AACT
aact = AACT (connectivity=connectivity, distanceMatrix=distanceMatrix, diameters=diameters, bondHardness=bondHardness, \
           electronegativity=electronegativity, netCharge=netCharge, maxOrder=maxOrder, chargeTransferTopology=chargeTransferTopology)
charges = aact.compute()

In [73]:
list(zip(atoms, np.around(aact.charges, 5)))

[('CChl', 0.36329),
 ('CL', -0.31369),
 ('CH2', -0.00039),
 ('CH2', -0.10515),
 ('CH2', 0.04603),
 ('CH2', 0.14666),
 ('CH3', -0.13676)]

In [29]:
pd.DataFrame(aact.bondJMatrix)

,0,1,2,3,4,5,6,7,8
0,28.330138,-4.044152,-4.044152,-3.896308,1.874232,0.000000,0.000000,0.000000,0.000000
1,-4.044152,28.330138,-4.044152,-3.896308,1.874232,0.000000,0.000000,0.000000,0.000000
2,-4.044152,-4.044152,28.330138,-3.896308,1.874232,0.000000,0.000000,0.000000,0.000000
3,-3.896308,-3.896308,-3.896308,14.174855,3.814912,-1.940680,0.000000,0.000000,0.000000
4,1.874232,1.874232,1.874232,3.814912,16.118641,3.814912,-1.874232,-1.874232,-1.874232
5,0.000000,0.000000,0.000000,-1.940680,3.814912,14.174855,4.037588,3.896308,3.896308
6,0.000000,0.000000,0.000000,0.000000,-1.874232,4.037588,39.105269,-4.185431,-4.185431
7,0.000000,0.000000,0.000000,0.000000,-1.874232,3.896308,-4.185431,28.330138,-4.044152
8,0.000000,0.000000,0.000000,0.000000,-1.874232,3.896308,-4.185431,-4.044152,28.330138


In [74]:
# SQE
sqe = SQE (connectivity=connectivity, distanceMatrix=distanceMatrix, diameters=diameters, hardness=hardness, bondHardness=bondHardness, \
           electronegativity=electronegativity, netCharge=netCharge, maxOrder=maxOrder, chargeTransferTopology=chargeTransferTopology, kappa=1, lam=1)
charges = sqe.compute()

In [75]:
list(zip(atoms, np.around(sqe.charges, 5)))

[('CChl', 0.12315),
 ('CL', -0.14447),
 ('CH2', 0.02452),
 ('CH2', -0.00867),
 ('CH2', 0.01581),
 ('CH2', 0.04778),
 ('CH3', -0.05814)]